# 1. Beschaffung des Wissens für den Chatbot-Prototyp

Bevor mit dem eigentlichen Chatbot begonnen werden kann, müssen die Inforamtionen auf deren Basis der Chatbot arbeiten soll beschafft und gespeichter werden. Im Rahmen der Arbeit werden dafür die Informationen wie Kontaktdaten und Forschungsschwerpunkte duech Web Scraping extrahiert und in eine sqlite Datenbank gespeichert. 

## Erstellung der Datenbank und Tabellen

In [65]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import time
from tqdm import tqdm

### Beschaffung von Namen der HHN Professoren 

Es werden Namen von HHN Professoren aus Websites gescraped und in eine Liste gespeichter, sodass diese spätter in Iterationen gescraped werden können. 

In [75]:
# get the names of the profs in the html. output is a list eith names
def get_prof_names():
    url = "https://www.meinprof.de/unis/baden-wuerttemberg/hs-heilbronn"
    page = requests.get(url).text
    doc = BeautifulSoup(page, "html.parser")
    
    names = []
    
    for i in range(284):
        a = doc.find_all(class_ = "bold")
        name = a[i].find("a").contents
        names.append(name)
    
    flat_list = [item for sublist in names for item in sublist]
    
    for i in range(len(flat_list)):
        flat_list[i] = str(flat_list[i])
    
    #remove profs with one name
    profs = []
    for i in range(len(flat_list)):
        if "," in flat_list[i]:
            profs.append(flat_list[i])
    return profs

In [76]:
# calling the function
names = get_prof_names()

In [78]:
# change the names so that they can be palced at the end of the url
def transform_name(list):
    prof_name_list = []
    for i in range(len(list)):
        prof_name = list[i]
        prof_name = str(prof_name)
        words = prof_name.split()
        words = words[1], words[0]
        ## joining the words and printing
        prof_name = " ".join(words).strip()
        prof_name = prof_name.replace(",","")
        prof_name = prof_name.replace(" ", ".")
        prof_name_list.append(prof_name)
    return prof_name_list

In [79]:
prof_names = transform_name(names) 

In [81]:
# manually created list with professor names
more_prof_names = ["carsten.lanquillon", "detlef.stern", "jochen.guenther", "helmut.beckmann", "thomas.schaeffer", "sonja.salmen", "ted.azarmi", "katharina.sperl", "rektor", "ulrich.brecht", "ruth.fleuchaus", "raoul.zoellner","christoph.schwerdtfeger"]

In [94]:
# concatenate the big list and the small manual list
all_prof_names=prof_names+more_prof_names
all_prof_names

['Christopher.Adams',
 'Uwe.Ahrens',
 'Tobias.Albrecht',
 'Wolfgang.Albrecht',
 'Martin.Alles',
 'Roland.Alter',
 'Patrick.Balve',
 'Frank.Bantle',
 'Jonathan.Bate',
 'Ralf.Bäuchl',
 'Hans.Bauer',
 'Hans.Bauer',
 'Rainer.Bayer',
 'Helmut.Beckmann',
 'Peter.Bender',
 'Rolf.Bendl',
 'Tomas.Benz',
 'Bleyel.Bernd',
 'Tobias.Bernecker',
 'Thomas.Bezold',
 'Axel.Birk',
 'Lutz.Blecher',
 'Peter.Blessing',
 'Rolf.Blumentritt',
 'Ralf.Bochert',
 'Bernd.Bracht',
 'Bernd.Bracht',
 'Terry.Braemer',
 'David.Braun',
 'Ulrich.Brecht',
 'Birgit.Brigl',
 'Georg.Bucher',
 'Jasmin.Buck',
 'Georg.Bühler',
 'Uwe.Burk',
 'August.Burr',
 'Franzi.Buscholl',
 'Constanze.Clarke',
 'Georg.Clauß',
 'Andreas.Daberkow',
 'Maren-Raphaela.Dechant',
 'Hartmut.Dickhaus',
 'Michael.Dietzel',
 'Ralf.Dillerup',
 'Heiko.Dirks',
 'Juergen.Doneit',
 'Babette.Dorner',
 'Manfred.Dorsch',
 'Michael.Drach',
 'Daniela.Eisele',
 'Wolfgang.Elmendorf',
 'Frank.Engel',
 'Anja.Engelmann',
 'Wolfgang.Ernst',
 'Simon.Fauser',
 'Uwe.Fede

## Funktion für Kontaktdaten der Professoren 

In [201]:
def get_prof_info(list):
    title_list = []
    first_name_list = []
    last_name_list = []
    telephone_list = []
    email_list = []
    office_list = []
    postal_address_list = []
    
    for prof in list:
        
        url = "https://www.hs-heilbronn.de/"+prof
        result = requests.get(url).text
        doc = BeautifulSoup(result, "html.parser")
        
        ### get title, first_name, last_name ### 
        full_name = doc.find(class_="mt-0 mb-3 p-0 sal-name").string
        
        # first_name
        first_name = full_name.split()[-2]
        first_name_list.append(first_name)
        
        # last_name
        last_name = full_name.split()[-1]
        last_name_list.append(last_name)
                
        # title
        title = full_name.replace(first_name, "")
        title = title.replace(last_name, "")
        title_list.append(title)
        
        ### get phone number ###
        telephone = doc.find(class_="mb-2").find(class_="color-teal").string
        telephone_list.append(telephone)
        
        ### get omly the first email ###
        email = doc.find_all(class_="mb-2")[1].find(class_="color-teal").string
        email_list.append(email)
        
        ### get office ###
        if "@" in doc.find_all(class_="mb-2")[2].span.string:
        # go to the next item to get the office
            office = doc.find_all(class_="mb-2")[3].span.string
        else:
            office = doc.find_all(class_="mb-2")[2].span.string
        office_list.append(office)
        
        ### get postal_address ###
        if len(doc.find_all(class_="mb-2")[3].span.string) < 10:
            postal_address = doc.find_all(class_="mb-2")[4].span.string
        else:
            postal_address = doc.find_all(class_="mb-2")[3].span.string
        postal_address_list.append(postal_address)
        
    df = pd.DataFrame({'title': title_list, 'first_name': first_name_list, 'last_name': last_name_list, 'telephone': telephone_list, 'email': email_list, 'office': office_list, 'postal_address': postal_address_list})

    return df

In [206]:
df = get_prof_info(all_prof_names)

AttributeError: 'NoneType' object has no attribute 'string'

In [203]:
df

,title,first_name,last_name,telephone,email,office,postal_address
0,Prof. Dr.-Ing.,Carsten,Lanquillon,+49 7131 504 6942,carsten.lanquillon@hs-heilbronn.de,S.3.45,"Max-Planck-Str. 39, 74081 Heilbronn"
1,Prof. Dr.,Detlef,Stern,+49 7131 504 450,detlef.stern@hs-heilbronn.de,S.3.44,"Max-Planck-Str. 39, 74081 Heilbronn"
2,Prof. Dr.-Ing.,Jochen,Günther,+49 (0) 7131 504 6945,jochen.guenther@hs-heilbronn.de,S.3.44,"Max-Planck-Str. 39, 74081 Heilbronn"
3,Prof. Dr.,Helmut,Beckmann,+49 7131 504 514,helmut.beckmann@hs-heilbronn.de,S.3.47,"Max-Planck-Str. 39, 74081 Heilbronn"
4,Dipl.-Inf.,Thomas,Schäffer,+49 7131 504 6699,thomas.schaeffer@hs-heilbronn.de,S.3.46,"Max-Planck-Str. 39, 74081 Heilbronn"
5,Prof. Dr.,Sonja-Maria,Salmen,+49 7131 504 477,sonja.salmen@hs-heilbronn.de,S.3.43,"Max-Planck-Str. 39, 74081 Heilbronn"
6,Prof. Dr.,Ted,Azarmi,+49 7131 504 6697,ted.azarmi@hs-heilbronn.de,N413,"Max-Planck-Str. 39, 74081 Heilbronn"
7,Dr.,Katharina,Sperl,+49 7131 504 519,katharina.sperl@hs-heilbronn.de,N540,"Max-Planck-Str. 39, 74081 Heilbronn"
8,Prof. Dr.-Ing.,Oliver,Lenzen,+49 7131 504 201,rektor@hs-heilbronn.de,T.3.26,"Max-Planck-Str. 39, 74081 Heilbronn"
9,Prof. Dr.,Ulrich,Brecht,+49 7131 504 551,ulrich.brecht@hs-heilbronn.de,T.3.22,"Max-Planck-Str. 39, 74081 Heilbronn"
